<a href="https://colab.research.google.com/github/universeatmyfeet/NLP/blob/master/Classwork/NLP_J051_17thDec_SentimentAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentAnalyzer
from nltk.classify.scikitlearn import SklearnClassifier
import textblob
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [72]:
!pip install glove_python

In [0]:
from glove import Corpus, Glove

In [138]:
db = pd.read_csv("https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv")
db.head()

,Topic,Sentiment,TweetId,TweetDate,TweetText
0,apple,positive,126415614616154112,Tue Oct 18 21:53:25 +0000 2011,Now all @Apple has to do is get swype on the i...
1,apple,positive,126404574230740992,Tue Oct 18 21:09:33 +0000 2011,@Apple will be adding more carrier support to ...
2,apple,positive,126402758403305474,Tue Oct 18 21:02:20 +0000 2011,Hilarious @youtube video - guy does a duet wit...
3,apple,positive,126397179614068736,Tue Oct 18 20:40:10 +0000 2011,@RIM you made it too easy for me to switch to ...
4,apple,positive,126395626979196928,Tue Oct 18 20:34:00 +0000 2011,I just realized that the reason I got into twi...


In [0]:
db2 = pd.read_json('http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',lines = True)#reading gz file from url

**1: Applying sentiment analyzers on both the datasets and determining polarity of the sentiment:** 

**For SIA of VADER (suited for tweets) :**



> ![Image of sentiment](https://miro.medium.com/max/656/1*G8yV2iaqqfaGfmRPRem2Fw.png)








**For TextBlob:**

> Polarity is a float value within the range [-1.0 to 1.0] where 0 indicates neutral, +1 indicates a very positive sentiment and -1 represents a very negative sentiment.

> Subjectivity is a float value within the range [0.0 to 1.0] where 0.0 is very objective and 1.0 is very subjective. Subjective sentence expresses some personal feelings, views, beliefs, opinions, allegations, desires, beliefs, suspicions, and speculations where as Objective sentences are factual.



In [0]:
def sentimentanalyzer(text):
  sid = SentimentIntensityAnalyzer()
  scores = sid.polarity_scores(text)
  
  if scores['compound'] >= 0.05 : 
        sent = 'Positive'    
        result = scores['pos']
  
  elif scores['compound'] <= - 0.05 : 
        sent = "Negative" 
        result = scores['neg']
  
  else : 
        sent = "Neutral"
        result = scores['neu']

  d = {"TextBlob_polarity":TextBlob(text).sentiment.polarity,"SIA score" : scores,"SIA max" : (sent,result)}      
  return d

In [0]:
db['TweetText_sentiment'] = db.TweetText.apply(sentimentanalyzer)

In [142]:
db['TweetText_sentiment'][1]

{'SIA max': ('Positive', 0.199),
 'SIA score': {'compound': 0.4549, 'neg': 0.0, 'neu': 0.801, 'pos': 0.199},
 'TextBlob_polarity': 0.5}

In [0]:
db2['reviewText_sentiment'] = db2.reviewText.apply(sentimentanalyzer)

In [117]:
db2['reviewText_sentiment'][1]

{'SIA max': ('Positive', 0.099),
 'SIA score': {'compound': 0.9963, 'neg': 0.029, 'neu': 0.872, 'pos': 0.099},
 'TextBlob_polarity': 0.17093871975019514}

**2. Comparison of analysed sentiment to sentiment in the twitter dataset**

In [143]:
db['Sentiment'].unique()

array(['positive', 'negative', 'neutral', 'irrelevant'], dtype=object)

In [0]:
db = db[db["Sentiment"] != "irrelevant"]

In [145]:
db['Sentiment']

0       positive
1       positive
2       positive
3       positive
4       positive
          ...   
4537     neutral
4538     neutral
4539     neutral
4540     neutral
4541     neutral
Name: Sentiment, Length: 3424, dtype: object

In [0]:
def comparison(dict):
  if dict["TextBlob_polarity"]>0:
    return 'positive'
  if dict["TextBlob_polarity"]<0:
    return 'negative'
  if dict["TextBlob_polarity"]==0:
    return 'neutral'

In [0]:
db["sentimentcheck"] = db.TweetText_sentiment.apply(comparison)

In [148]:
db["sentimentcheck"] 

0        neutral
1       positive
2       positive
3       positive
4       positive
          ...   
4537    positive
4538     neutral
4539     neutral
4540    negative
4541     neutral
Name: sentimentcheck, Length: 3424, dtype: object

In [0]:
db['Sentiment'] = pd.Categorical(db['Sentiment'])

db['sentimentcheck'] = pd.Categorical(db['sentimentcheck'])

In [149]:

cm = confusion_matrix(db["sentimentcheck"],db["Sentiment"])
print("Confusion Matrix - TextBlob:")
print(cm)

Confusion Matrix - TextBlob:
[[ 234  201   38]
 [ 171 1213  124]
 [ 167  919  357]]


In [150]:
from sklearn.metrics import classification_report
print(classification_report(db["sentimentcheck"],db["Sentiment"]))

              precision    recall  f1-score   support

    negative       0.41      0.49      0.45       473
     neutral       0.52      0.80      0.63      1508
    positive       0.69      0.25      0.36      1443

    accuracy                           0.53      3424
   macro avg       0.54      0.52      0.48      3424
weighted avg       0.58      0.53      0.49      3424

